In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [16]:
url_inicial = 'http://www.revistagastroenterologiamexico.org/es-numeros-anteriores-2010'
url_root = 'http://www.revistagastroenterologiamexico.org/es-numeros-anteriores-2010'
r=requests.get(url_inicial)

In [17]:
soup = BeautifulSoup(r.text, 'html.parser')

In [18]:
box = soup.find('div', class_='sumarios decada-2010')
volumen=box.findAll('div',class_='sumario row')
vol = [x.find('a').get('href')for x in volumen] 
vol = [urljoin (url_root, i) for i in vol]
vol 

['http://www.revistagastroenterologiamexico.org/es-vol-75-num-4-sumario-X0375090610X96845',
 'http://www.revistagastroenterologiamexico.org/es-vol-75-num-3-sumario-X0375090610X87927',
 'http://www.revistagastroenterologiamexico.org/es-vol-75-num-2-sumario-X0375090610X84785',
 'http://www.revistagastroenterologiamexico.org/es-vol-75-num-1-sumario-X0375090610X81995']

In [19]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='collFull') 
    volumen1=box1.findAll('div', class_='item')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol1 = [urljoin (url_root, i) for i in vol1]
    vol2 = [urljoin (url_root, i) for i in vol2]
    vol2+=vol1

In [20]:
vol2

['http://www.revistagastroenterologiamexico.org/es-utilidad-goma-mascar-resolucion-del-articulo-X0375090610898397',
 'http://www.revistagastroenterologiamexico.org/es-nivel-satisfaccion-pacientes-tras-un-articulo-X0375090610898401',
 'http://www.revistagastroenterologiamexico.org/es-entrenamiento-diseccion-endoscopica-submucosa-mediante-articulo-X037509061089841X',
 'http://www.revistagastroenterologiamexico.org/es-evaluacion-indices-no-invasivos-fibrosis-articulo-X0375090610898428',
 'http://www.revistagastroenterologiamexico.org/es-capsaicin-induction-esophageal-symptoms-in-articulo-X0375090610898436',
 'http://www.revistagastroenterologiamexico.org/es-intolerancia-fructosa-pacientes-con-sindrome-articulo-X0375090610898444',
 'http://www.revistagastroenterologiamexico.org/es-diferencias-entre-clasificaciones-chicago-tradicional-articulo-X0375090610898452',
 'http://www.revistagastroenterologiamexico.org/es-validez-externa-un-modelo-experimental-articulo-X0375090610898460',
 'http://w

In [21]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='collFull') 
    volumen1=box1.findAll('div', class_='item')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [22]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina http://www.revistagastroenterologiamexico.org/es-numeros-anteriores-2010


In [23]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

91

In [24]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [33]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista de Gastroenterología de México'
    a='III. Medicina y Ciencias de la Salud'
    tem='Gastroenterología'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('div', class_='elsevierItemTitulo').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='elsevierItemTextoCompletoTexto').get_text(strip=True)
        content_book['Resumen']=resu.replace("�� Caso clínico","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
        linkart = [urljoin (url_root, i) for i in linkart]
    #link articulo
    try:
        link=s_item.find('div', class_='right').find('a').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
        link = [urljoin (url_root, i) for i in link]
    return content_book

In [34]:
list_scraper=list_scraper[0:101]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [35]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Utilidad de la goma de mascar para la resoluci...,?? IntroducciónLa reducción de la motilidad in...,None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...
1,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Nivel de satisfacción de pacientes tras un pro...,?? AntecedentesLa endoscopia gastrointestinal ...,None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...
2,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Entrenamiento en la disección endoscópica de l...,¿ IntroducciónLos simuladores han sido ampliam...,None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...
3,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Evaluación de índices no invasivos de fibrosis...,?? IntroducciónLa hepatitis crónica por virus ...,None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...
4,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Capsaicin induction of esophageal symptoms in ...,¿ IntroductionHeartburn is a very common sympt...,None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...
...,...,...,...,...,...,...,...,...
86,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Ensayo clínico: el efecto de una leche ferment...,"Simrén M, Ömhan L, Olsson J,et al.Clinical tri...",None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...
87,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Novedad editorial: Escrutinio en enfermedades ...,"Uscanga L, Bernal-Reyes R, editores. Escrutini...",None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...
88,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Novedad editorial: Gastroenterología y hepatol...,"García-Compeán D, Maldonado-Garza H, editores....",None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...
89,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,¿Está indicado buscar infección porHelicobacte...,ProsEn una era en la que la incidencia de cánc...,None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...


In [36]:
df_catalogo.to_csv('Revista019.13.csv', index=False)